In [1]:
import numpy as np
import pandas as pd

from math import *
from sklearn.cluster import DBSCAN

pd.options.display.max_rows = 5
pd.options.display.max_columns = 50

In [2]:
cd ..

/Users/mgcrp/Documents/GitHub/hse_graphvis_2021


In [3]:
cd data

/Users/mgcrp/Documents/GitHub/hse_graphvis_2021/data


In [4]:
files = [
    '0228685-200613084148143.csv',
    '0228687-200613084148143.csv',
    '0228689-200613084148143.csv',
    '0228693-200613084148143.csv',
    '0228696-200613084148143.csv',
    '0228699-200613084148143.csv'
]

df = pd.read_csv(files[0], sep='\t')

for file in files[1:]:
    df = df.append( pd.read_csv(file, sep='\t') )

/Users/mgcrp/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14,17,36,38,39,40,41,44,46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/mgcrp/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (17,38,39,40,41,46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/mgcrp/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10,38,39,40,41,46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
def calc_dist(x1, y1, x2, y2):
    return sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))


def flight_by_bird(bird):
    a = [[]]
    
    #кластеры для каждого месяца
    for i in range(1, 13):
        df_one_species = df[(df['species'] == bird) & (df['month'] == i)][['decimalLatitude', 'decimalLongitude']]
        if df_one_species.shape[0] > 0:
            clustering = DBSCAN(eps=2, min_samples=20).fit(df_one_species)
            for j in range(clustering.labels_.max()+1):
                a.append([df_one_species[clustering.labels_ == j]['decimalLatitude'].mean(), df_one_species[clustering.labels_ == j]['decimalLongitude'].mean(), i])
    monthly_clusters = np.zeros((len(a) - 1, 5))
    monthly_clusters[:,:3] = np.array(a[1:])
    
    #кластеры кластеров
    clustering = DBSCAN(eps=5, min_samples=3).fit(monthly_clusters[:,:2])
    
    #переводим месяца в координаты (одна координата от января от июля, другая от октября до апреля)
    for i in range(monthly_clusters.shape[0]):
        month = monthly_clusters[i,2]
        if month <= 7:
            monthly_clusters[i,3] = month-1
        else:
            monthly_clusters[i,3] = 13-month
        if month <= 4:
            monthly_clusters[i,4] = month+2
        elif month <= 10:
            monthly_clusters[i,4] = 10-month
        else:
            monthly_clusters[i,4] = month-10
    
    #Средние координаты месяцев
    clusters = [[monthly_clusters[clustering.labels_ == j, 3].mean(), monthly_clusters[clustering.labels_ == j, 4].mean()] for j in range(clustering.labels_.max()+1)]
    
    #Находим 2 самых далёких по средему месяцу кластера
    first_cl = -1
    second_cl = -1
    time_dist = 0
    for i in range(len(clusters)):
        for j in range(i+1, len(clusters)):
            d = calc_dist(clusters[i][0], clusters[i][1], clusters[j][0], clusters[j][1])
            if (d > time_dist):
                time_dist = d
                first_cl = i
                second_cl = j
    
    #Если кластеры разнесены по времени, то перелётная, инчае нет. Для перелётных возвращаем серединки 2 кластеров, найденных в предыдущем пунке, для не перелётных 2 раза середину по всем
    if time_dist >= 2.5:
        return bird,\
            'Перелётная',\
            monthly_clusters[clustering.labels_ == first_cl, 0].mean(),\
            monthly_clusters[clustering.labels_ == first_cl, 1].mean(),\
            monthly_clusters[clustering.labels_ == second_cl, 0].mean(),\
            monthly_clusters[clustering.labels_ == second_cl, 1].mean(),\
            monthly_clusters[clustering.labels_ == first_cl, 0],\
            monthly_clusters[clustering.labels_ == first_cl, 1],\
            monthly_clusters[clustering.labels_ == second_cl, 0],\
            monthly_clusters[clustering.labels_ == second_cl, 1]
    else:
        return bird,\
            'Не перелётная',\
            monthly_clusters[:,0].mean(),\
            monthly_clusters[:,1].mean(),\
            monthly_clusters[:,0].mean(),\
            monthly_clusters[:,1].mean()

In [11]:
from tqdm import tqdm

df1 = df.groupby('species')[['gbifID', 'individualCount']].agg({'gbifID' : 'count', 'individualCount' : 'sum'})
species = list(df1[df1['gbifID'] > 1000].index)

out = [[]]

for bird in tqdm( species, total=len(species) ):
    r = flight_by_bird(bird)
    if (r != None):
        out.append(list(r))
#     print(len(out)-1)

out = out[1:]
out

100%|██████████| 114/114 [06:28<00:00,  3.41s/it]


[['Acanthis flammea',
  'Перелётная',
  51.84071105254674,
  5.517069124113475,
  61.4026028842466,
  16.566326644120135,
  array([51.97315902, 52.03977273, 51.50920141]),
  array([5.79238687, 5.69659091, 5.06222959]),
  array([59.24424118, 59.04810585, 59.43610617, 59.66475434, 57.92241785,
         64.81770591, 57.57744532, 65.62717154, 61.28501151, 65.74529744,
         63.79138895, 57.60906485, 67.88560907, 64.52956605, 58.6506139 ,
         61.46086772, 59.90488993, 61.04659433]),
  array([16.32430831, 16.38544516, 15.99227092, 15.44485673, 13.21799614,
         20.81047757, 12.84896187, 17.06118369, 14.52285764, 23.70116176,
         13.61062519, 13.30171365, 19.32183007, 21.29126065, 14.15158498,
         17.7964092 , 15.90994533, 16.50099075])],
 ['Accipiter gentilis',
  'Не перелётная',
  56.44352589612512,
  11.595519573960997,
  56.44352589612512,
  11.595519573960997],
 ['Aegolius funereus',
  'Не перелётная',
  62.72261396096462,
  17.889487183745196,
  62.72261396096462,


In [12]:
migrating = []

for item in out:
    if item[1] == 'Перелётная':
        migrating.append(item)

In [20]:
from scipy.spatial import ConvexHull

for i in range(len(migrating)):
#     print(
#         np.array([ [x, y] for x, y in zip(migrating[i][7], migrating[i][6])])
#     )
    migrating[i].append(
        ConvexHull( np.array([ [x, y] for x, y in zip(migrating[i][7], migrating[i][6])]) ).vertices
    )
    migrating[i].append(
        ConvexHull( np.array([ [x, y] for x, y in zip(migrating[i][9], migrating[i][8])]) ).vertices
    )

In [21]:
migrating[1]

['Anas acuta',
 'Перелётная',
 57.98947527433644,
 15.085319855134994,
 37.148088731025645,
 -7.224148453589744,
 array([57.14636467, 57.0635668 , 58.08270806, 58.97630486, 59.69772376,
        56.76675043, 56.71719924, 61.625351  , 56.81997515, 57.3664454 ,
        60.89952505, 58.16540969, 57.83381434, 57.45621916, 56.68060264,
        56.53364414]),
 array([14.44865676, 14.57033323, 15.4450551 , 15.85024272, 16.79260812,
        16.86014712, 12.62252936, 14.3549287 , 16.74967426, 13.07103536,
        15.64984205, 15.61079625, 15.2164721 , 15.48803952, 14.40727804,
        14.22747898]),
 array([37.18578432, 37.13095045, 37.12753142]),
 array([-7.52811088, -7.18835425, -6.95598023]),
 array([ 7,  6, 15,  5,  4, 10], dtype=int32),
 array([2, 0, 1], dtype=int32)]

In [28]:
migrating[1][10], migrating[1][11]

(array([ 7,  6, 15,  5,  4, 10], dtype=int32), array([2, 0, 1], dtype=int32))

In [39]:
poly_out = []

for item in migrating:
    _tmp_from = []
    _tmp_to = []
    
    for i in item[10]:
        _tmp_from.append( f'{ round(item[7][i], 4) } { round(item[6][i], 4) }' )
    _tmp_from.append(_tmp_from[0])
        
    for i in item[11]:
        _tmp_to.append( f'{ round(item[9][i], 4) } { round(item[8][i], 4) }' )
    _tmp_to.append(_tmp_to[0])

    poly_out.append([item[0], f'POLYGON(({",".join(_tmp_from)}))'])
    poly_out.append([item[0], f'POLYGON(({",".join(_tmp_to)}))'])

In [40]:
df_polyOut = pd.DataFrame(np.array(poly_out), columns=['Name', 'Poly'])

In [41]:
df_polyOut.to_csv('polygons_for_carto.csv', index=False)

In [44]:
line_out = []

for item in migrating:
    line_out.append([
        item[0],
        f"LINESTRING({item[3]} {item[2]},{item[5]} {item[4]})"
    ]
    )

In [45]:
line_out[1]

['Anas acuta',
 'LINESTRING(15.085319855134994 57.98947527433644,-7.224148453589744 37.148088731025645)']

In [46]:
df_lineOut = pd.DataFrame(np.array(line_out), columns=['Name', 'Poly'])

In [48]:
df_lineOut.to_csv('lines_for_carto.csv', index=False)

In [ ]:
df_out = pd.DataFrame(np.array(out), columns = ['Name', 'Type', 'Start_Latitude', 'Start_Longitude', 'Finish_Latitude', 'Finish_Longitude'])

In [ ]:
df_out.to_excel('birds.xlsx', index=False)